In [5]:
import cv2
import math
import numpy as np
import time

camera = cv2.VideoCapture(0)

cx=0
cy=0
while True:    
    success, frame = camera.read()  
    if success:    
        height, width = frame.shape[0:2]  
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    
        bin1 = cv2.inRange(hsv, (17, 158, 0), (35, 255, 255)) # желтый
        bin2 = cv2.inRange(hsv, (0, 158, 0), (16, 255, 255)) # красный
        bin3 = cv2.inRange(hsv, (36, 158, 0), (52, 255, 255)) # зеленый
        bin4 = cv2.inRange(hsv, (66, 158, 0), (108, 255, 255)) # голубой
        binary = bin1+bin2+bin3+bin4  
        roi = cv2.bitwise_and(frame, frame, mask=binary)   
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_NONE) 
        for i,cnt in enumerate(contours):
            if len(contours) != 0:  
                maxc = max(contours, key=cv2.contourArea)  
                moments = cv2.moments(cnt) 
                if moments["m00"] > 20: 
                    cx = int(moments["m10"] / moments["m00"])  
                    cy = int(moments["m01"] / moments["m00"])  
                    cv2.drawContours(frame, maxc, -1, (0, 255, 0), 2) 
                    cv2.line(frame, (cx, 0), (cx, height), (0, 255, 0), 2)  
                    cv2.line(frame, (0, cy), (width, cy), (0, 255, 0), 2)  
            cv2.putText(frame, 'Axes x: {}'.format(cx), (width - 500, height - 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
            cv2.putText(frame, 'Axes y: {}'.format(cy), (width - 500, height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
        cv2.imshow('frame', frame)  # выводим все кадры на экран
        cv2.waitKey(1)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [2]:
import cv2
import math
import numpy as np
import time

# Параметры калибровки камеры (должны быть получены из предыдущей калибровки)
ret = True  # Успешность калибровки
mtx = np.array([[800.,   0., 320.],  # Матрица камеры (Получаем в результате калибровки)
                [  0., 800., 240.],
                [  0.,   0.,   1.]])

dist = np.array([[0.1, -0.05, 0.001, 0.002, 0.0]])  # Коэффициенты дисторсии (Получаем в результате калибровки)

camera = cv2.VideoCapture(0)

# Проверяем, открылась ли камера
if not camera.isOpened():
    print("Ошибка: Не удалось открыть камеру!")
    exit()

cx = 0
cy = 0

print("Запуск обработки видео. Нажмите 'q' для выхода.")

while True:    
    success, frame = camera.read()  
    
    if not success:
        print("Ошибка: Не удалось получить кадр с камеры!")
        break
    
    # Коррекция искажений камеры
    if ret:
        h, w = frame.shape[:2]
        # Получаем новую матрицу камеры с учетом обрезки
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
        
        # Исправляем искажения
        frame_undistorted = cv2.undistort(frame, mtx, dist, None, newcameramtx)
        
        # Обрезаем изображение по ROI
        x, y, w, h = roi
        frame_undistorted = frame_undistorted[y:y+h, x:x+w]
        frame = frame_undistorted
       
    # Обработка цветов
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    
    bin1 = cv2.inRange(hsv, (17, 158, 0), (35, 255, 255)) # желтый
    bin2 = cv2.inRange(hsv, (0, 158, 0), (16, 255, 255)) # красный
    bin3 = cv2.inRange(hsv, (36, 158, 0), (52, 255, 255)) # зеленый
    bin4 = cv2.inRange(hsv, (66, 158, 0), (108, 255, 255)) # голубой
    binary = bin1 + bin2 + bin3 + bin4  
    
    roi = cv2.bitwise_and(frame, frame, mask=binary)   
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    if len(contours) > 0:  
        maxc = max(contours, key=cv2.contourArea)  
        moments = cv2.moments(maxc) 
        
        if moments["m00"] > 20: 
            cx = int(moments["m10"] / moments["m00"])  
            cy = int(moments["m01"] / moments["m00"])  
            
            # Рисуем контур и оси
            cv2.drawContours(frame, [maxc], -1, (0, 255, 0), 2) 
            cv2.line(frame, (cx, 0), (cx, h), (0, 255, 0), 2)  
            cv2.line(frame, (0, cy), (w, cy), (0, 255, 0), 2)  
            
            # Отображаем центра
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
    # Отображаем координаты
    cv2.putText(frame, 'X: {}'.format(cx), (10, h - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
    cv2.putText(frame, 'Y: {}'.format(cy), (10, h - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('Original Frame', frame)
    
    # Выход по нажатию 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождаем ресурсы
camera.release()
cv2.destroyAllWindows()

Запуск обработки видео. Нажмите 'q' для выхода.


In [1]:
import cv2
import math
import numpy as np
import time

# Параметры калибровки камеры (должны быть получены из предыдущей калибровки)
ret = True  # Успешность калибровки
mtx = np.array([[800.,   0., 320.],  # Матрица камеры (Получаем в результате калибровки)
                [  0., 800., 240.],
                [  0.,   0.,   1.]])

dist = np.array([[0.1, -0.05, 0.001, 0.002, 0.0]])  # Коэффициенты дисторсии (Получаем в результате калибровки)

camera = cv2.VideoCapture(0)

# Проверяем, открылась ли камера
if not camera.isOpened():
    print("Ошибка: Не удалось открыть камеру!")
    exit()

# Переменные для стабилизации
cx = 0
cy = 0
prev_cx = 0
prev_cy = 0
stable_count = 0
MIN_STABLE_FRAMES = 5  # Минимальное количество стабильных кадров
MAX_MOVEMENT = 15      # Максимальное допустимое перемещение между кадрами

# Параметры фильтрации контуров
MIN_CONTOUR_AREA = 500  # Минимальная площадь контура
MIN_CONTOUR_WIDTH = 20  # Минимальная ширина контура
MIN_CONTOUR_HEIGHT = 20 # Минимальная высота контура

print("Запуск обработки видео. Нажмите 'q' для выхода.")

while True:    
    success, frame = camera.read()  
    
    if not success:
        print("Ошибка: Не удалось получить кадр с камеры!")
        break
    
    # Коррекция искажений камеры
    if ret:
        h, w = frame.shape[:2]
        # Получаем новую матрицу камеры с учетом обрезки
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
        
        # Исправляем искажения
        frame_undistorted = cv2.undistort(frame, mtx, dist, None, newcameramtx)
        
        # Обрезаем изображение по ROI
        x, y, w, h = roi
        frame_undistorted = frame_undistorted[y:y+h, x:x+w]
        frame = frame_undistorted
    
    # Создаем копию для обработки
    processed_frame = frame.copy()
       
    # Обработка цветов
    hsv = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2HSV)    
    
    # Более точные диапазоны цветов
    bin1 = cv2.inRange(hsv, (20, 100, 100), (30, 255, 255)) # желтый
    bin2 = cv2.inRange(hsv, (156, 82, 92), (212, 255, 255)) # красный (нижний диапазон)
    bin3 = cv2.inRange(hsv, (35, 100, 100), (85, 255, 255)) # зеленый
    bin4 = cv2.inRange(hsv, (90, 100, 100), (130, 255, 255)) # голубой/синий
    
    binary = bin1 + bin2 + bin3 + bin4  
    
    # Морфологические операции для улучшения качества маски
    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)  # Заполнение маленьких отверстий
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)   # Удаление маленьких шумов
    
    # Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Фильтрация контуров по площади и размеру
    valid_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA:
            continue
            
        # Проверяем размер bounding box
        x, y, contour_w, contour_h = cv2.boundingRect(contour)
        if contour_w < MIN_CONTOUR_WIDTH or contour_h < MIN_CONTOUR_HEIGHT:
            continue
            
        valid_contours.append(contour)
    
    # Выбор самого большого валидного контура
    if valid_contours:
        largest_contour = max(valid_contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)
        
        if moments["m00"] > 0: 
            new_cx = int(moments["m10"] / moments["m00"])  
            new_cy = int(moments["m01"] / moments["m00"])
            
            # Проверка стабильности положения
            movement = math.sqrt((new_cx - prev_cx)**2 + (new_cy - prev_cy)**2)
            
            if movement < MAX_MOVEMENT:
                stable_count += 1
            else:
                stable_count = 0
            
            # Обновляем координаты только если положение стабильно
            if stable_count >= MIN_STABLE_FRAMES:
                cx, cy = new_cx, new_cy
            
            prev_cx, prev_cy = new_cx, new_cy
            
            # Рисуем контур
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 3)          
            
            # Отображаем центр
            cv2.circle(frame, (cx, cy), 8, (0, 0, 255), -1)
            cv2.circle(frame, (cx, cy), 12, (255, 255, 255), 2)
        
    
    cv2.putText(frame, f'X: {cx}', (10, h - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
    cv2.putText(frame, f'Y: {cy}', (10, h - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('Original Frame', frame)
    
    # Выход по нажатию 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождаем ресурсы
camera.release()
cv2.destroyAllWindows()

Запуск обработки видео. Нажмите 'q' для выхода.


KeyboardInterrupt: 